In [1]:
import pandas as pd
import numpy as np
import os
import sys
from pathlib import Path
import json
import wave
# sudo apt install portaudio19-dev python3-pyaudio
import pyaudio
import torchaudio 

In [2]:
data_root_dir = Path('/home/scc/Downloads/speech2textData')
data_dir = data_root_dir.joinpath('genshin-voice-v3.5-mandarin/data')
label_path = data_root_dir.joinpath('result.json')

In [3]:
with label_path.open(mode='r') as f:
    label_res = json.load(f)

In [4]:
list(label_res.keys())[:10]

['00000ad748e4defa',
 '00003b433f092c26',
 '00003e876928ec1f',
 '0000519d3a5004e8',
 '0000b124af136a62',
 '0000cb0ced63faaa',
 '0000d5d3e270abc5',
 '0000d6fff0f3a1de',
 '0000f9f7ada1c623',
 '00010b7659af6235']

In [5]:
file = [i for i in data_dir.iterdir()][0]
print(file)
df = pd.read_parquet(file)

/home/scc/Downloads/speech2textData/genshin-voice-v3.5-mandarin/data/train-00038-of-00067-dc30261178723032.parquet


In [6]:
df[df.type == 'AnimatorEvent']['audio'].iloc[0]['bytes']

b'RIFF\xa4\xe2\x00\x00WAVEfmt \x10\x00\x00\x00\x01\x00\x01\x00\x80\xbb\x00\x00\x00w\x01\x00\x02\x00\x10\x00data\x80\xe2\x00\x00\xf8\xff\xeb\xff\xe0\xff\xf1\xff\xe6\xff\xf7\xff\xec\xff\xfd\xff\xfa\xff\x13\x00\x04\x00\x1d\x00\x0e\x00\x17\x00\x18\x00\x1d\x00\x14\x00\x19\x00\n\x00\x07\x00\x00\x00\xff\xff\xf4\xff\xf3\xff\xec\xff\xef\xff\xec\xff\xef\xff\xf4\xff\xf6\xff\xff\xff\x03\x00\x0e\x00\x12\x00\x1b\x00\x1a\x00\x1b\x00\x17\x00\x16\x00\x16\x00\x15\x00\x16\x00\x10\x00\t\x00\xfb\xff\xf4\xff\xf0\xff\xeb\xff\xeb\xff\xe8\xff\xe8\xff\xe9\xff\xeb\xff\xf2\xff\xfc\xff\x07\x00\x0b\x00\x0e\x00\x0f\x00\x15\x00\x11\x00\x0f\x00\x0c\x00\x0e\x00\x11\x00\x12\x00\x0c\x00\r\x00\x0b\x00\x10\x00\x0e\x00\x0f\x00\x0b\x00\x0c\x00\x03\x00\x00\x00\xf7\xff\xf4\xff\xeb\xff\xf0\xff\xe7\xff\xec\xff\xeb\xff\xf4\xff\xf7\xff\xfc\xff\xfe\xff\x07\x00\x0b\x00\x10\x00\x0e\x00\t\x00\x01\x00\xfa\xff\xee\xff\xe3\xff\xdb\xff\xd5\xff\xcf\xff\xce\xff\xce\xff\xd0\xff\xd7\xff\xe1\xff\xf0\xff\xfe\xff\x0e\x00\x1d\x00)\x003\x00;\x00@\

In [10]:
# yesno_data = torchaudio.datasets.LIBRISPEECH('.', download=True)

In [2]:
yesno_data = torchaudio.datasets.LIBRISPEECH('.', download=False)

yesno_data[0]

(tensor([[-0.0065, -0.0055, -0.0062,  ...,  0.0033,  0.0005, -0.0095]]),
 16000,
 'CHAPTER ONE MISSUS RACHEL LYNDE IS SURPRISED MISSUS RACHEL LYNDE LIVED JUST WHERE THE AVONLEA MAIN ROAD DIPPED DOWN INTO A LITTLE HOLLOW FRINGED WITH ALDERS AND LADIES EARDROPS AND TRAVERSED BY A BROOK',
 103,
 1240,
 0)

In [4]:
yesno_data[0][0].shape

torch.Size([1, 225360])